In [1]:
import pandas as pd
from pathlib import Path

In [2]:
from config import *
import psycopg2

In [3]:
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port= PORT)
cursor = conn.cursor()
cursor.execute("Select * FROM results LIMIT 0")
colnames = [desc[0] for desc in cursor.description]

In [4]:
cursor.execute("SELECT * FROM results")
data = cursor.fetchall()
results_df = pd.DataFrame(data, columns=colnames)
results_df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [5]:
cursor.execute("Select * FROM races LIMIT 0")
colnames = [desc[0] for desc in cursor.description]

In [6]:
cursor.execute("SELECT * FROM races")
data = cursor.fetchall()
races_df = pd.DataFrame(data, columns=colnames)
races_df.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [7]:
# Read results.csv and races.csv 
# results_df = pd.read_csv(Path('../Resources/Dataset/results.csv'))
# races_df = pd.read_csv(Path('../Resources/Dataset/races.csv'))

In [8]:
# Drop columns we dont need
results_df = results_df.drop(columns =['number', 'positionText', 'positionOrder', 
                                       'points', 'laps', 'time', 'milliseconds', 
                                       'fastestLap', 'rank', 'fastestLapTime',
                                      'fastestLapSpeed'])                      

In [9]:
# Rename certain columns to make them easier to understand
results_df = results_df.rename(columns={"position": "ending_position", 
                                        "grid": "starting_position",
                                        "statusId": "finishing_status" })                 

In [10]:
print(results_df.shape)
results_df.head()

(25040, 7)


,resultId,raceId,driverId,constructorId,starting_position,ending_position,finishing_status
0,1,18,1,1,1,1,1
1,2,18,2,2,5,2,1
2,3,18,3,3,7,3,1
3,4,18,4,4,11,4,1
4,5,18,5,1,3,5,1


In [11]:
# Merge results_df with races_df on raceId. Only bring in columns from races_df that we need
results_df = (results_df.merge(races_df, left_on='raceId', right_on='raceId')
          .reindex(columns=['resultId', 'raceId', 'driverId', 'constructorId',
                                  'starting_position', 'ending_position', 'finishing_status', 
                                 'year', 'circuitId', 'name']))

In [12]:
print(results_df.shape)
results_df.head()

(25040, 10)


,resultId,raceId,driverId,constructorId,starting_position,ending_position,finishing_status,year,circuitId,name
0,1,18,1,1,1,1,1,2008,1,Australian Grand Prix
1,2,18,2,2,5,2,1,2008,1,Australian Grand Prix
2,3,18,3,3,7,3,1,2008,1,Australian Grand Prix
3,4,18,4,4,11,4,1,2008,1,Australian Grand Prix
4,5,18,5,1,3,5,1,2008,1,Australian Grand Prix


These represent finishing statuses. Drivers can finish multiple laps down from the lead driver. For our purpose, we only care that the driver finished the race so we will convert all these statuses to 1. 

1: Finished, 11: +1 Lap, 12: +2 Laps, 13: +3 Laps, 14: +4 Laps,15: +5 Laps, 16: +6 Laps,17: +7 Laps,
18: +8 Laps, 19: +9 Laps, 45: +11 Laps, 50: +17 Laps,12: +42 Laps, 53: +13 Laps,55: +12 Laps, 
58: +26 Laps, 88: +10 Laps, 111: +14 Laps, 112: +15 Laps, 113: +25 Laps, 114: +18 Laps, 115: +22 Laps,
116: +16 Laps, 117: +24 Laps, 118: +29 Laps, 119: +23 Laps, 120: +21 Laps, 122: +44 Laps, 123: +30 Laps, 
124: +19 Laps, 125: +46 Laps,127: +20 Laps,133: +49 Laps,134: +38 Laps

In [13]:
# We will finish all the statuses where drivers finish the race to 1
replace = {
    1:1, 11:1, 12:1, 13:1, 14:1, 15:1, 16:1, 17:1, 18:1, 19:1, 45:1, 50:1, 128:1, 53:1, 55:1, 58:1, 
    88:1, 111:1, 112:1, 113:1, 114:1, 115:1, 116:1, 117:1, 118:1, 119:1, 120:1, 122:1, 123:1, 124:1, 
    125:1, 127:1, 133:1, 134:1  
}
results_df["finishing_status"] = results_df["finishing_status"].replace(replace)

In [14]:
# Transform finish_status column. Anything which is not 1 (driver did not finish race) is converted to 2
def change_status(x):
    if x == 1:
        return 1
    else:
        return 2
    
results_df["finishing_status"] = results_df["finishing_status"].apply(change_status)
print(results_df.shape)
results_df.head()

(25040, 10)


,resultId,raceId,driverId,constructorId,starting_position,ending_position,finishing_status,year,circuitId,name
0,1,18,1,1,1,1,1,2008,1,Australian Grand Prix
1,2,18,2,2,5,2,1,2008,1,Australian Grand Prix
2,3,18,3,3,7,3,1,2008,1,Australian Grand Prix
3,4,18,4,4,11,4,1,2008,1,Australian Grand Prix
4,5,18,5,1,3,5,1,2008,1,Australian Grand Prix


In [15]:
# Identify unique ending_positions to identify if any rows need to be dropped
results_df["ending_position"].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '\\N', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33'],
      dtype=object)

In [16]:
# Replace all "\\N" ending_positions to 0 
replace1 = {"\\N":0}
results_df["ending_position"] = results_df["ending_position"].replace(replace1)

In [17]:
# Create a train_test column. 2019 data will be test and 2010 to 2018 will be 

results_df['train_test'] = results_df['year']

def change_year(x):
    if x < 2010:
        return 3
    elif x == 2019:
        return 2 
    elif x > 2019:
        return 4
    else:
        return 1
results_df["train_test"] = results_df["train_test"].apply(change_year)

print(results_df.shape)
results_df.head()

(25040, 11)


,resultId,raceId,driverId,constructorId,starting_position,ending_position,finishing_status,year,circuitId,name,train_test
0,1,18,1,1,1,1,1,2008,1,Australian Grand Prix,3
1,2,18,2,2,5,2,1,2008,1,Australian Grand Prix,3
2,3,18,3,3,7,3,1,2008,1,Australian Grand Prix,3
3,4,18,4,4,11,4,1,2008,1,Australian Grand Prix,3
4,5,18,5,1,3,5,1,2008,1,Australian Grand Prix,3


In [18]:
# Rearrainging columns
results_df = results_df[['resultId',
                         'raceId',
                         'name',
                         'year',
                         'circuitId',
                         'driverId',
                         'constructorId',
                         'starting_position',
                         'finishing_status',
                         'train_test',
                         'ending_position']]
print(results_df.shape)
results_df.head()

(25040, 11)


,resultId,raceId,name,year,circuitId,driverId,constructorId,starting_position,finishing_status,train_test,ending_position
0,1,18,Australian Grand Prix,2008,1,1,1,1,1,3,1
1,2,18,Australian Grand Prix,2008,1,2,2,5,1,3,2
2,3,18,Australian Grand Prix,2008,1,3,3,7,1,3,3
3,4,18,Australian Grand Prix,2008,1,4,4,11,1,3,4
4,5,18,Australian Grand Prix,2008,1,5,1,3,1,3,5


In [19]:
# Creating the final dataframe with data between 2010 to 2019
results_final_df = results_df.loc[results_df["train_test"] <= 2]
print(results_final_df.shape)
results_final_df.head()

(4297, 11)


,resultId,raceId,name,year,circuitId,driverId,constructorId,starting_position,finishing_status,train_test,ending_position
20323,20323,337,Bahrain Grand Prix,2010,3,4,6,3,1,1,1
20324,20324,337,Bahrain Grand Prix,2010,3,13,6,2,1,1,2
20325,20325,337,Bahrain Grand Prix,2010,3,1,1,4,1,1,3
20326,20326,337,Bahrain Grand Prix,2010,3,20,9,1,1,1,4
20327,20327,337,Bahrain Grand Prix,2010,3,3,131,5,1,1,5


In [20]:
# Extract CSV file
# results_final_df.to_csv('../Resources/PythonExport/results_final.csv')

In [21]:
connect = f"postgres://postgres:{PASSWORD}@{HOST}:{PORT}/postgres"

In [22]:
from sqlalchemy import create_engine

In [23]:
engine = create_engine(connect)

In [24]:
results_final_df.to_sql(name='results_final', con=engine, if_exists='replace')